# Homework 3: Supervised Learning

Due Date: Monday, May 22, 2023 at Midnight PST (Submit this .ipynb file on Canvas)

Your name: **Add your name here**

<span style="color:red">100 total points</span>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Logistic Regression</a></span><ul class="toc-item"><li><span><a href="#Describe-Logistic-Regression" data-toc-modified-id="Describe-Logistic-Regression-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Describe Logistic Regression</a></span></li><li><span><a href="#Generate-a-Simple-Train/Test-Split" data-toc-modified-id="Generate-a-Simple-Train/Test-Split-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Generate a Simple Train/Test Split</a></span></li><li><span><a href="#Run-Logistic-Regression-&amp;-Evaluate-Performance" data-toc-modified-id="Run-Logistic-Regression-&amp;-Evaluate-Performance-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Run Logistic Regression &amp; Evaluate Performance</a></span></li><li><span><a href="#Evaluate-the-Most-Important-Genes" data-toc-modified-id="Evaluate-the-Most-Important-Genes-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Evaluate the Most Important Genes</a></span></li><li><span><a href="#Regularization" data-toc-modified-id="Regularization-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Regularization</a></span><ul class="toc-item"><li><span><a href="#Discuss-regularization" data-toc-modified-id="Discuss-regularization-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Discuss regularization</a></span></li><li><span><a href="#Vary-the-Regularization-and-Re-evaluate-Performance" data-toc-modified-id="Vary-the-Regularization-and-Re-evaluate-Performance-1.5.2"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>Vary the Regularization and Re-evaluate Performance</a></span></li></ul></li><li><span><a href="#PR-and-ROC-Curves" data-toc-modified-id="PR-and-ROC-Curves-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>PR and ROC Curves</a></span></li><li><span><a href="#Use-Bootstrapping-to-Obtain-Confidence-Intervals" data-toc-modified-id="Use-Bootstrapping-to-Obtain-Confidence-Intervals-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Use Bootstrapping to Obtain Confidence Intervals</a></span><ul class="toc-item"><li><span><a href="#Bootstrapping-Performance" data-toc-modified-id="Bootstrapping-Performance-1.7.1"><span class="toc-item-num">1.7.1&nbsp;&nbsp;</span>Bootstrapping Performance</a></span></li><li><span><a href="#Bootstrapping-Gene-Coefficients" data-toc-modified-id="Bootstrapping-Gene-Coefficients-1.7.2"><span class="toc-item-num">1.7.2&nbsp;&nbsp;</span>Bootstrapping Gene Coefficients</a></span></li></ul></li></ul></li><li><span><a href="#Decision-Trees" data-toc-modified-id="Decision-Trees-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Decision Trees</a></span><ul class="toc-item"><li><span><a href="#Summarize-Decision-Trees" data-toc-modified-id="Summarize-Decision-Trees-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Summarize Decision Trees</a></span></li><li><span><a href="#Make-a-Train/Test-Split-For-Growth-Rate-Determination" data-toc-modified-id="Make-a-Train/Test-Split-For-Growth-Rate-Determination-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Make a Train/Test Split For Growth Rate Determination</a></span></li><li><span><a href="#Train-a-Regression-Tree-&amp;-Evaluate-Performance" data-toc-modified-id="Train-a-Regression-Tree-&amp;-Evaluate-Performance-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Train a Regression Tree &amp; Evaluate Performance</a></span></li><li><span><a href="#Show-the-Tree-Structure-&amp;-Comment-on-Top-Genes" data-toc-modified-id="Show-the-Tree-Structure-&amp;-Comment-on-Top-Genes-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Show the Tree Structure &amp; Comment on Top Genes</a></span></li></ul></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn import model_selection, linear_model, metrics, tree

# read in the data
file_name = 'Homework_Data.xlsx'

data = pd.read_excel(file_name, 
                     sheet_name = 'Expression Data',
                     index_col = 0)
sample_table = pd.read_excel(file_name,
                             sheet_name = 'Experimental Conditions',
                             index_col = 0)
gene_table = pd.read_excel(file_name,
                           sheet_name = 'Gene Information',
                           index_col = 0)

# normalize the data
data_norm = pd.DataFrame(stats.zscore(data, axis = 1), 
                         columns = data.columns, index = data.index)

## Logistic Regression

Two different strains were used to generate the homework dataset: E. coli MG1655 and BW25113. They are closely related, so the difference is subtle -- let's see if you can train a logistic regression model to predict the strain given an expression profile. 

### Describe Logistic Regression

Summarize logistic regression. Be sure to discuss classification vs. regression, training and testing data, gene coefficients, and the logistic function. 

<span style="color:red">5 points</span>

### Generate a Simple Train/Test Split

The line of code provided below adds a column to the sample_table which is true if the sample is BW25113 and false if the sample is MG1655. This will be the label you are trying to predict. Note that we have a class imbalance problem here; there are 72 MG1655 samples and 30 BW25113 samples.

Split the samples into a train group and a test group. Make sure that:

- The train group is approximately 3 times larger than the test group (a test size of 25% of the data)
- Both groups contain approximately equal proportions of both trace element mixture samples.
- You always keep samples that belong to the exact same condition together. This is important, because replicates from the same condition being split into the different groups will allow the algorithm to see very similar data in both the train and test group. The purpose of a train/test split is to ensure you aren't overfitting your model, and splitting up replicates would compromise that.
- If you want everything to be consistent across multiple runs, specify a random_state.

Also, you will need the 'is_BW' labels for both groups. You could create two pd.Series (train_labels & test_labels) where the index is the list of samples that obeys the rules above, and the values are from the 'is_BW' column of the sample_table.

**Hint:** You could use the following steps to do this problem:

1. Get a list of samples from the boolean is_BW column for both True and False
2. Get the unique conditions ('Condition ID') associated with those samples
3. Randomly split both sets of conditions into a train (75%) and test (25%) group
4. Combine your two test groups into a single test group with both strains, and do the same with your train groups
5. Convert from conditions back into samples (pd.Series.isin() might help)
6. Make the labels structures by indexing back into the sample_table

<span style="color:red">5 points</span>

In [2]:
sample_table['is_BW'] = sample_table['Strain Description'].str.contains('BW25113')

### Run Logistic Regression & Evaluate Performance

Run logistic regression with the train/test split you made above. [Use normalized data](https://scikit-learn.org/stable/modules/preprocessing.html). Be sure to specify no regularization penalty, as we will explore that in a later problem. 

Obtain predicted labels for your testing set, and then make a contingency table comparing those labels to the true labels. Quantify: accuracy, sensitivity, specificity, precision, F1 score, and Matthews correlation coefficient. Be sure each number is labeled in your output. Comment on the values: Did your test perform well? Which is a better overall metric for performance and why?

<span style="color:red">10 points</span>

### Evaluate the Most Important Genes

Look at the 5 genes that are most important to your model. Do you think these make sense as ways of discerning between the strains, or are they more likely the result of other underlying variables, like the specific samples that happened to be in the training set?

The strain differences are shown in [this table](http://bioinfo.ccs.usherbrooke.ca/BW25113.html).

<span style="color:red">5 points</span>

### Regularization

#### Discuss regularization

1. What is regularization and what is its goal?
2. What is the difference between L1 and L2 regularization? 
    1. What do they each do to the distribution of coefficients? 
    2. Why would you use one or the other?

<span style="color:red">5 points</span>

#### Vary the Regularization and Re-evaluate Performance

Vary the L1 and L2 regularization strengths and retrain your model, then evaluate each regularization with a good metric from your answer to problem 1.3. Visualize your results and comment on them -- what is the overall effect of regularization in this case, and is your metric more sensitive to one type of regularization over the other?

What specific values of the parameters would you recommend using based on this analysis?

<span style="color:red">10 points</span>

### PR and ROC Curves

The prediction algorithm of logistic regression works like this:

1. Obtain the dot product of the learned weights and sample's features
2. Transform the linear value with a logistic function to obtain a probability p
3. Threshold the probability p to make a True/False call

PR and ROC curves are designed to help place the threshold assigned in step 3. You can obtain the probabilites from step 2 using .predict_proba().

Create PR and ROC curves for your **unregularized** model. (If you are curious, try it with your regularized model, too). Also plot a histogram of your probabilities to help you interpret them. Be sure to include the area under the ROC curve. Explain the shape of the curves and comment on the quality of your model. What value should be used for the threshold?

<span style="color:red">10 points</span>

### Use Bootstrapping to Obtain Confidence Intervals

Bootstrapping can be used to obtain value distributions for your performance and feature importance across a large random sample of possible train/test splits.

#### Bootstrapping Performance

Bootstrap your train/test split and run logistic regression 100x. Make sure you still keep replicates from the same condition together. Use your recommended regularization parameters from 1.5.2. Each time, save the six performance measures for the testing data from problem 1.3 as well as the gene coefficient vector. 

Obtain medians and 90% confidence intervals for all performance metric results. Visualize them in a table or plot. Comment on them -- is there a lot of variation based on the conditions chosen for the training data? Did your bootstrapped models perform better or worse than your initial train/test split?

<span style="color:red">10 points</span>

#### Bootstrapping Gene Coefficients

Perform median and 90% confidence interval calculations on your gene coefficients. Look at the top 5 genes (by their medians) and discuss them. Are these related to the actual problem, or are they more affected by other underlying variables?

<span style="color:red">5 points</span>

## Decision Trees

Decision trees make great classifiers, but they can also be used as regression trees to predict a scalar value. Let's make a regression tree that predicts growth rate based on gene expression.

### Summarize Decision Trees

Summarize the following:

1. What is a decision tree, and how is it traversed?
2. How is it made?
3. The difference between a classification decision tree and a regression tree
4. Strengths and weaknesses as a method
5. The following regularization methods:
    1. Boosting/early stopping
    2. Bagging/random forests
    
<span style="color:red">10 points</span>

### Make a Train/Test Split For Growth Rate Determination

Remove the samples that are not annotated with growth rate. Then, make a 4:1 Train/Test split of the remaining **conditions** as you did for problem 1.2 (except with no need to make a distinction between classes, and a larger train to test ratio due to the smaller amount of available data).

<span style="color:red">5 points</span>

### Train a Regression Tree & Evaluate Performance

Train and test a Decision Tree Regressor using the split you just defined. Create a scatter plot comparing the predicted values with the actual ones. Also quantify the mean squared error, mean absolute error, explained variance, and R^2 coefficient of determination between your predicted and true growth rates.

<span style="color:red">10 points</span>

### Show the Tree Structure & Comment on Top Genes

Visualize the tree structure. Comment on it and at least its top three genes. It may be helpful to name the features by gene_name. How would you traverse it to get to the lowest possible growth rates? The highest possible growth rates?

<span style="color:red">10 points</span>